Experiments

In [1]:
import math
import pandas as pd
import altair as alt
import numpy as np
import flammkuchen as fl

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

from spinorama.graph import graph_params_default, graph_freq, graph_spinorama
from spinorama.load import filter_graphs, graph_melt, load_normalize
from spinorama.compute_normalize import normalize_mean

df_all = fl.load("../cache.parse_all_speakers.h5")

In [2]:
df_klippel = df_all["K"]["Kanto YU6"]["ASR"]["asr"]

In [3]:
pir = df_klippel["Estimated In-Room Response_unmelted"]
spin = df_klippel["CEA2034_unmelted"]
freq = pir.Freq.values
spin_mean = df_klippel["CEA2034_original_mean"]
# pir['Estimated In-Room Response'] -= spin_mean
spin["Listening Window"] -= spin_mean

In [4]:
params = graph_params_default.copy()
params["ymin"] = -5
params["ymax"] = 5

In [5]:
from spinorama.auto_target import get_freq, get_target

In [6]:
optim_config = {
    "curve_names": ["Listening Window"],
    "freq_reg_min": 20,
    "freq_reg_max": 20000,
    "target_min_freq": 20,
    "target_max_freq": 20000,
    "slope_listening_window": -6,
}
freq = get_freq(df_klippel, optim_config)
lw_target = get_target(spin, freq[1], "Listening Window", optim_config) + 36

In [7]:
df_lw_target = pd.DataFrame({"Freq": freq[1], "Target": lw_target})
graph_freq(graph_melt(spin[{"Freq", "Listening Window"}]), params) + alt.Chart(df_lw_target).mark_line().encode(
    x="Freq:Q", y="Target:Q"
)

alt.LayerChart(...)

In [8]:
first_freq = 0
for i, f in enumerate(freq[1]):
    if f >= 100:  # Hz
        first_freq = i
        break
last_freq = -1
for i, f in enumerate(reversed(freq[1])):
    if f <= 16000:  # Hz
        last_freq = -i + 1
        break

target = spin["Listening Window"].values - lw_target

kernel_size = 10
kernel = np.ones(kernel_size) / kernel_size
target_smoothed = np.convolve(target, kernel, mode="same")

df_target = pd.DataFrame(
    {
        "Freq": freq[1][first_freq:last_freq],
        "data": target[first_freq:last_freq],
        "smoothed": target_smoothed[first_freq:last_freq],
    }
)

In [9]:
graph_target = (
    alt.Chart(graph_melt(df_target))
    .mark_line(size=3, point=True)
    .encode(
        x=alt.X("Freq:Q", scale=alt.Scale(type="log", nice=False, domain=[100, 16000])),
        y=alt.Y("dB:Q", scale=alt.Scale(zero=False, nice=False, domain=[-5, 5]), title="dB"),
        color=alt.Color("Measurements"),
    )
    .properties(width=1024)
)

In [10]:
starget = target_smoothed[first_freq:last_freq]
n = len(starget)


def areas(data, begin, end):
    stored = np.zeros((n, n))
    for i in range(begin, end):
        stored[i][i + 1 :] = [np.sum(data[i:j]) for j in range(i + 1, end)]
    return stored


stored = areas(starget, 0, n)

In [11]:
df = pd.DataFrame(
    {
        "value": starget,
        "sign": np.diff(np.r_[False, starget < 0]).cumsum(),
    }
)

In [12]:
areas_init = df.groupby("sign").apply(lambda x: np.trapz(x.value))

In [13]:
areas_init

sign
1     -0.262574
2     10.922347
3    -37.765722
4    110.424899
5      0.000000
dtype: float64

In [14]:
# recursive on size of array?
def partition4(starget_len):
    best_c = -1
    best = 0
    best_v = None
    for i in range(0, starget_len - 4):
        for j in range(i + 1, starget_len - 3):
            for k in range(j + 1, starget_len - 2):
                cur_v = [
                    stored[0][i],
                    stored[i][j],
                    stored[j][k],
                    stored[k][starget_len - 1],
                ]
                cur = np.sum(np.abs(cur_v))
                if cur > best:
                    best = cur
                    best_c = (i, j, k)
                    best_v = cur_v
    return best, best_c, best_v


def partition4f(starget_len):
    best_c = -1
    best = 0
    best_v = None
    for i in range(0, starget_len - 4):
        for i2 in range(i, starget_len - 4):
            for j in range(i2 + 1, starget_len - 3):
                for k in range(j + 1, starget_len - 2):
                    cur_v = [
                        stored[i][i2],
                        stored[i2][j],
                        stored[j][k],
                        stored[k][starget_len - 1],
                    ]
                    cur = np.sum(np.abs(cur_v))
                    if cur > best:
                        best = cur
                        best_c = (i, i2, j, k)
                        best_v = cur_v
    return best, best_c, best_v


def partition5(starget_len):
    best_c = -1
    best = 0
    best_v = None
    for i in range(0, starget_len - 5):
        for j in range(i + 1, starget_len - 4):
            for k in range(j + 1, starget_len - 3):
                for l in range(k + 1, starget_len - 2):
                    cur_v = [
                        stored[0][i],
                        stored[i][j],
                        stored[j][k],
                        stored[k][l],
                        stored[l][starget_len - 1],
                    ]
                    cur = np.sum(np.abs(cur_v))
                    if cur > best:
                        best = cur
                        best_c = (i, j, k)
                        best_v = cur_v
    return best, best_c, best_v


best, best_c, best_v = partition4f(len(starget))

print(best, best_c, best_v)

160.14159288741143 (0, 2, 21, 61) [-0.5251475824404834, 11.051773460081003, -37.79927140954132, 110.76540043534864]


In [15]:
df_zones = pd.DataFrame(
    {
        "x": np.repeat([freq[1][first_freq + i] for i in best_c], 3),
        "y": np.tile([-5, 5, -5], len(best_c)),
    }
)

In [16]:
graph_target + alt.Chart(df_zones).mark_line().encode(x=alt.X("x:Q"), y=alt.Y("y:Q"), color=alt.value("green"))

alt.LayerChart(...)